In [2]:
import sys
!{sys.executable} -m pip install --upgrade pip
!{sys.executable} -m pip install face-recognition
!{sys.executable} -m pip install opencv-contrib-python-headless

print("\n Installation complete!")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 23.0 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
  Using cached face_recognition-1.3.0-py2.py3-none-any.whl.metadata (21 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.1/100.1 MB 42.9 MB/s  0:00:02
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for face-recognition-models: filename=face_recognition_models-0.3.0-py2.py3-none-any.whl size=100566240 sha256=027f4bf5b9918e6fda52cf72b1fd2514748a6703c1b090d8936c51c4fc1dcdb6
  Stored in directory: /root/.cache/pip/wheels/8f/47/c8/f44c5aebb7507f7c8a2c0bd23151d732d0f0bd6884ad4ac635
Successfully built face-recognition-models
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [face-recognition]
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 MB 87.8 MB/s  0:00:00

 In

In [3]:
import cv2
import face_recognition
import numpy as np
import os
from datetime import datetime
import csv
from IPython.display import display
import pandas as pd
from google.colab.patches import cv2_imshow
import matplotlib.pyplot as plt
%matplotlib inline

print("All Libraries imported successfully!")

All Libraries imported successfully!


In [5]:
class FaceAttendanceSystem:
    def __init__(self):
        self.known_face_encodings = []
        self.known_face_names = []
        self.attendance_marked = set()
        self.attendance_file = 'attendance.csv'

    def load_known_faces(self, images_path='known_faces'):
        """Load and encode faces"""
        if not os.path.exists(images_path):
            print(f"❌ Folder '{images_path}' not found!")
            return

        print("Loading known faces...")
        for filename in os.listdir(images_path):
            if filename.endswith(('.jpg', '.jpeg', '.png')):
                name = os.path.splitext(filename)[0]
                image_path = os.path.join(images_path, filename)

                image = face_recognition.load_image_file(image_path)
                encodings = face_recognition.face_encodings(image)

                if encodings:
                    self.known_face_encodings.append(encodings[0])
                    self.known_face_names.append(name)
                    print(f"✓ Loaded: {name}")

        print(f"\nTotal faces loaded: {len(self.known_face_names)}")

    def mark_attendance(self, name):
        """Mark attendance"""
        if name not in self.attendance_marked:
            self.attendance_marked.add(name)
            now = datetime.now()
            date_string = now.strftime('%Y-%m-%d')
            time_string = now.strftime('%H:%M:%S')

            file_exists = os.path.isfile(self.attendance_file)

            with open(self.attendance_file, 'a', newline='') as f:
                writer = csv.writer(f)
                if not file_exists:
                    writer.writerow(['Name', 'Date', 'Time'])
                writer.writerow([name, date_string, time_string])

            print(f"✓ Attendance: {name} at {time_string}")
            return True
        return False

    def recognize_from_image(self, image_path):
        """Recognize faces from image"""
        if not self.known_face_encodings:
            print("❌ No known faces loaded!")
            return

        print(f"Processing: {image_path}")

        image = face_recognition.load_image_file(image_path)
        face_locations = face_recognition.face_locations(image)
        face_encodings = face_recognition.face_encodings(image, face_locations)

        image_cv = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        print(f"Found {len(face_locations)} face(s)")

        for (top, right, bottom, left), face_encoding in zip(face_locations, face_encodings):
            matches = face_recognition.compare_faces(self.known_face_encodings, face_encoding)
            name = "Unknown"

            face_distances = face_recognition.face_distance(self.known_face_encodings, face_encoding)
            if len(face_distances) > 0:
                best_match_index = np.argmin(face_distances)
                if matches[best_match_index]:
                    name = self.known_face_names[best_match_index]
                    self.mark_attendance(name)

            color = (0, 255, 0) if name != "Unknown" else (0, 0, 255)
            cv2.rectangle(image_cv, (left, top), (right, bottom), color, 3)
            cv2.rectangle(image_cv, (left, bottom - 35), (right, bottom), color, cv2.FILLED)
            cv2.putText(image_cv, name, (left + 6, bottom - 6),
                       cv2.FONT_HERSHEY_DUPLEX, 1, (255, 255, 255), 2)

        cv2_imshow(image_cv)

    def view_attendance(self):
        """View attendance records"""
        if not os.path.exists(self.attendance_file):
            print("No attendance records!")
            return

        df = pd.read_csv(self.attendance_file)
        print(f"\n{'='*60}")
        print("ATTENDANCE RECORDS")
        print(f"{'='*60}")
        display(df)
        print(f"\nTotal Records: {len(df)}")

    def reset_attendance(self):
        """Reset attendance for current session"""
        self.attendance_marked.clear()
        print("✓ Attendance reset for current session")

print("✓ Class defined successfully!")

✓ Class defined successfully!


In [6]:
# Cell 4: Initialize System
print("="*60)
print("FACE RECOGNITION ATTENDANCE SYSTEM")
print("="*60)

system = FaceAttendanceSystem()

FACE RECOGNITION ATTENDANCE SYSTEM


In [7]:
# Create system instance
system = FaceAttendanceSystem()

In [8]:
# Cell 5: Load Known Faces
system.load_known_faces('known_faces')

❌ Folder 'known_faces' not found!


In [10]:
# Cell 6: Display Loaded Faces (Optional)
#system.display_known_faces()

In [13]:
# Cell 7: Option 1 - Recognize from Webcam
# Runs for 60 seconds by default, change duration as needed
#system.recognize_faces_webcam(duration=60)

In [14]:
# Cell 8: Option 2 - Recognize from Image File
# Upload or specify an image path
# system.recognize_from_image('test_image.jpg')

In [15]:
# Cell 9: View Attendance Records
system.view_attendance()

No attendance records!


In [16]:
# Cell 10: Reset Attendance (if needed)
# system.reset_attendance()